In [1]:
# importando libs
import pandas as pd
from sqlalchemy import create_engine, text
import psycopg2
import boto3
from io import StringIO
from botocore.exceptions import ClientError
import datetime
import numpy as np

In [ ]:
usuario = "postgres"
senha = ""
host = "bd-relacional.cjeco8miq291.us-east-1.rds.amazonaws.com"
porta = 5432
banco = "postgres"

# Cria a engine de conexão
engine = create_engine(f"postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}")

In [3]:
def test_connection(engine):
    try:
        with engine.connect() as connection:
            # Testa a versão do PostgreSQL
            result = connection.execute(text("SELECT version();"))
            versao = result.fetchone()
            print("✅ Conectado com sucesso:", versao[0])

            # Lista as tabelas no schema público
            result = connection.execute(text("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public';
            """))
            tabelas = result.fetchall()
            print("📄 Tabelas no banco:")
            for tabela in tabelas:
                print("  -", tabela[0])

    except Exception as e:
        print("❌ Erro ao executar comandos:", e)

In [4]:
test_connection(engine)

✅ Conectado com sucesso: PostgreSQL 17.4 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 12.4.0, 64-bit
📄 Tabelas no banco:
  - dim_cliente
  - fato_pedidos
  - dim_produto


### Insert dos dados

In [5]:
ddl_dim_fato = ["""

--Criação das tabelas do modelo dimensional
                CREATE TABLE IF NOT EXISTS dim_cliente (
                id_cliente INT PRIMARY KEY,
                nome VARCHAR(100),
                idade INT,
                cidade VARCHAR(100)
                );
                
                CREATE TABLE IF NOT EXISTS dim_produto (
                id_produto INT PRIMARY KEY,
                nome_produto VARCHAR(100),
                categoria VARCHAR(50),
                preco DECIMAL(10,2)
                );

                CREATE TABLE IF NOT EXISTS fato_pedidos (
                id_pedido INT PRIMARY KEY,
                id_cliente INT REFERENCES dim_cliente(id_cliente),
                id_produto INT REFERENCES dim_produto(id_produto),
                data_pedido DATE,
                quantidade INT,
                valor_total DECIMAL(10,2)
                )
"""]

# Executar a criação das tabelas
with engine.begin() as conn:
  for cmd in ddl_dim_fato:
    conn.execute(text(cmd))
  print("Tabelas criadas!")
  

Tabelas criadas!


In [6]:
ddl_dim_fato = ["""

INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (1, 'Daniela da Cunha', 58, 'Dias');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (2, 'Marcos Vinicius da Mota', 25, 'Costela');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (3, 'Vitor Nascimento', 19, 'da Paz');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (4, 'Dra. Caroline Nascimento', 65, 'Mendes da Prata');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (5, 'Igor Rezende', 35, 'Nunes');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (6, 'Caroline Caldeira', 33, 'Fogaça');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (7, 'Heloísa da Mata', 32, 'Lima');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (8, 'Sofia da Luz', 26, 'Cardoso');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (9, 'Thales Castro', 65, 'Nascimento de Minas');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (10, 'Dra. Maria Teixeira', 24, 'Sales');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (11, 'Srta. Luna Costa', 61, 'Moreira');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (12, 'Antônio da Rosa', 65, 'Novaes');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (13, 'Ana Laura Rodrigues', 52, 'Nunes');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (14, 'Sr. Thiago Freitas', 23, 'Moura do Campo');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (15, 'Vitor Gabriel Melo', 55, 'das Neves Verde');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (16, 'Luiz Gustavo da Conceição', 45, 'da Rocha');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (17, 'Srta. Ana Carolina Ferreira', 20, 'das Neves');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (18, 'Luiz Gustavo Souza', 19, 'Monteiro');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (19, 'Bárbara Sales', 23, 'Duarte');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (20, 'Agatha Martins', 31, 'Souza');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (21, 'Davi Lucas Ferreira', 32, 'da Mata');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (22, 'Luna Ribeiro', 50, 'Novaes');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (23, 'João Miguel Fernandes', 56, 'Moura de Silva');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (24, 'Guilherme Farias', 19, 'Castro');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (25, 'Vitor Hugo Azevedo', 53, 'da Cunha');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (26, 'Lucas Nascimento', 30, 'Nunes de Mendes');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (27, 'Nina da Cruz', 63, 'Silveira Verde');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (28, 'Bianca Oliveira', 59, 'da Luz de Cardoso');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (29, 'Bernardo Fernandes', 62, 'Farias de Viana');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (30, 'Juan da Conceição', 52, 'Sales Grande');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (31, 'Dra. Manuela da Conceição', 44, 'Teixeira da Mata');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (32, 'Felipe Lopes', 32, 'Ferreira Verde');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (33, 'Erick Farias', 46, 'Azevedo Grande');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (34, 'Ana Laura Moura', 55, 'Nascimento');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (35, 'Bárbara Farias', 35, 'Oliveira do Oeste');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (36, 'Thomas das Neves', 69, 'Silva');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (37, 'Sr. Bryan Vieira', 18, 'da Cunha de Ribeiro');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (38, 'Sr. João Gabriel Santos', 66, 'da Mata');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (39, 'Mariana Lima', 69, 'Barros do Galho');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (40, 'Dr. Rodrigo Alves', 28, 'Ribeiro do Norte');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (41, 'Juan Novaes', 62, 'Viana');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (42, 'João Guilherme da Paz', 45, 'Cardoso das Pedras');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (43, 'Lorena Aragão', 39, 'Carvalho da Prata');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (44, 'Júlia Viana', 35, 'Mendes');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (45, 'João Miguel Novaes', 27, 'Campos');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (46, 'Caroline das Neves', 31, 'Silveira Verde');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (47, 'Breno Martins', 66, 'Silva da Praia');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (48, 'Benjamin Jesus', 39, 'Silveira');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (49, 'Isis Monteiro', 24, 'Araújo');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (50, 'Felipe Cardoso', 23, 'Fogaça da Serra');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (51, 'Ana Clara Nascimento', 42, 'Moreira de Caldeira');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (52, 'Maria Barbosa', 24, 'Martins de Dias');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (53, 'Vicente Silveira', 40, 'Dias de Ramos');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (54, 'Nicolas Barros', 40, 'Silva');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (55, 'Raquel Santos', 56, 'da Costa');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (56, 'Lucas Gabriel Teixeira', 34, 'Ferreira do Galho');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (57, 'Luiza das Neves', 69, 'Melo Paulista');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (58, 'Calebe Vieira', 20, 'Rocha');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (59, 'Luiza Duarte', 64, 'Teixeira');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (60, 'Ana Sophia Nascimento', 47, 'da Rocha');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (61, 'Júlia Farias', 52, 'Teixeira');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (62, 'Dr. Noah Cunha', 25, 'das Neves');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (63, 'Gabrielly Rocha', 42, 'Porto');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (64, 'Luana Barbosa', 23, 'Barros Verde');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (65, 'Srta. Amanda Ramos', 53, 'Freitas');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (66, 'Evelyn da Mota', 36, 'Jesus');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (67, 'Lorena Porto', 58, 'Martins da Praia');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (68, 'Valentina Carvalho', 57, 'Viana');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (69, 'Maysa Pereira', 41, 'Costela do Campo');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (70, 'Sr. Anthony Campos', 54, 'Martins de Novaes');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (71, 'Srta. Ana Carolina Nogueira', 30, 'Monteiro');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (72, 'Thales Moreira', 63, 'Nogueira Alegre');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (73, 'Luigi Porto', 22, 'Sales de da Rosa');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (74, 'Arthur Rezende', 20, 'Porto');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (75, 'Sophia Campos', 60, 'Silva');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (76, 'Srta. Kamilly Sales', 32, 'da Luz');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (77, 'Maria da Rosa', 67, 'das Neves do Amparo');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (78, 'Ana Beatriz Melo', 36, 'da Paz');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (79, 'Emilly Gonçalves', 23, 'Nascimento do Norte');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (80, 'João Aragão', 32, 'Almeida');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (81, 'Stella Caldeira', 24, 'Ribeiro do Sul');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (82, 'Ana Lívia Peixoto', 42, 'Lopes');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (83, 'Francisco Cunha', 35, 'da Costa');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (84, 'Francisco Rezende', 47, 'da Costa');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (85, 'Sr. Fernando Santos', 58, 'Barros');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (86, 'Ana Júlia Nascimento', 41, 'Araújo');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (87, 'Sr. Davi Luiz Moraes', 28, 'Nunes');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (88, 'Juan Nunes', 41, 'Almeida Grande');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (89, 'Ana Clara Rocha', 40, 'Fernandes');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (90, 'Maria Alice Nascimento', 31, 'da Rocha');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (91, 'Juan Moraes', 60, 'Rodrigues Verde');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (92, 'Isis da Mata', 35, 'Teixeira das Pedras');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (93, 'Sra. Emilly Nunes', 62, 'da Luz do Norte');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (94, 'Thiago Duarte', 61, 'Nascimento');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (95, 'Emanuella Fogaça', 59, 'Lopes do Norte');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (96, 'Lavínia Lima', 22, 'Dias');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (97, 'Bianca Fernandes', 56, 'Rodrigues de Ribeiro');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (98, 'Pedro Moreira', 58, 'Vieira Paulista');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (99, 'Raul Azevedo', 28, 'da Mata');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (100, 'Dr. Pedro Miguel Porto', 52, 'Barbosa');
"""]

# Executar a criação das tabelas
with engine.begin() as conn:
  for cmd in ddl_dim_fato:
    conn.execute(text(cmd))
  print("Dados inseridos!")
  

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "dim_cliente_pkey"
DETAIL:  Key (id_cliente)=(1) already exists.

[SQL: 

INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (1, 'Daniela da Cunha', 58, 'Dias');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (2, 'Marcos Vinicius da Mota', 25, 'Costela');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (3, 'Vitor Nascimento', 19, 'da Paz');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (4, 'Dra. Caroline Nascimento', 65, 'Mendes da Prata');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (5, 'Igor Rezende', 35, 'Nunes');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (6, 'Caroline Caldeira', 33, 'Fogaça');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (7, 'Heloísa da Mata', 32, 'Lima');
INSERT INTO dim_cliente (id_cliente, nome, idade, cidade) VALUES (8, 'Sofia da Luz', 

In [ ]:
ddl_dim_fato = ["""

INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (1, 'Iste Illum', 'Alimentos', 171.77);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (2, 'Libero Enim', 'Roupas', 277.25);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (3, 'Quis At', 'Livros', 227.42);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (4, 'Soluta At', 'Bebidas', 844.42);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (5, 'Cupiditate Nostrum', 'Eletrônicos', 236.76);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (6, 'Sunt Voluptas', 'Eletrônicos', 807.0);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (7, 'Rerum Explicabo', 'Roupas', 275.06);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (8, 'Iste Ipsum', 'Alimentos', 914.0);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (9, 'Sed Reprehenderit', 'Livros', 877.6);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (10, 'Quo Culpa', 'Bebidas', 220.5);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (11, 'Similique Facilis', 'Roupas', 401.68);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (12, 'Quo Facere', 'Roupas', 151.44);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (13, 'Ipsam Eum', 'Alimentos', 254.16);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (14, 'Soluta Atque', 'Livros', 543.59);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (15, 'Ducimus Molestias', 'Livros', 434.15);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (16, 'Reiciendis Cumque', 'Livros', 405.41);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (17, 'Doloribus Culpa', 'Alimentos', 997.35);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (18, 'Maiores Quaerat', 'Alimentos', 514.43);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (19, 'Vel Esse', 'Eletrônicos', 758.22);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (20, 'Earum Nostrum', 'Eletrônicos', 161.31);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (21, 'Suscipit Dolorum', 'Alimentos', 794.16);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (22, 'Atque Labore', 'Roupas', 600.44);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (23, 'Aut Eaque', 'Roupas', 387.8);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (24, 'Delectus Illum', 'Roupas', 533.82);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (25, 'Maiores Dicta', 'Livros', 862.17);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (26, 'Officiis Sint', 'Eletrônicos', 683.48);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (27, 'Voluptas Ipsam', 'Eletrônicos', 684.89);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (28, 'Sequi Aliquid', 'Livros', 753.37);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (29, 'Maiores Sunt', 'Bebidas', 120.44);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (30, 'Ipsum Rem', 'Roupas', 166.58);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (31, 'Maiores Aut', 'Eletrônicos', 954.28);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (32, 'Ratione Ad', 'Bebidas', 972.17);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (33, 'Quo Dolorem', 'Alimentos', 512.6);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (34, 'Quae Eveniet', 'Eletrônicos', 871.81);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (35, 'Voluptatum Iste', 'Bebidas', 843.25);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (36, 'Hic Corporis', 'Livros', 612.88);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (37, 'In Vero', 'Alimentos', 380.17);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (38, 'Id Sapiente', 'Alimentos', 543.99);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (39, 'Minus Distinctio', 'Livros', 919.31);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (40, 'Sed Quibusdam', 'Livros', 330.91);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (41, 'Rem Dolores', 'Eletrônicos', 120.75);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (42, 'Soluta Fugit', 'Bebidas', 879.93);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (43, 'Velit Nulla', 'Bebidas', 247.06);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (44, 'Harum Error', 'Alimentos', 879.23);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (45, 'Deserunt Explicabo', 'Eletrônicos', 94.8);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (46, 'Sequi Omnis', 'Roupas', 817.86);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (47, 'Aut Quos', 'Livros', 768.18);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (48, 'Eveniet Tempora', 'Alimentos', 663.15);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (49, 'Esse Molestias', 'Livros', 173.48);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (50, 'Modi Earum', 'Livros', 873.71);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (51, 'Sunt Natus', 'Roupas', 964.72);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (52, 'Amet Ducimus', 'Livros', 757.71);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (53, 'Sed Vero', 'Alimentos', 715.82);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (54, 'Nesciunt Sunt', 'Roupas', 995.2);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (55, 'Vero Molestias', 'Bebidas', 443.72);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (56, 'Eius Quod', 'Livros', 456.97);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (57, 'Repellendus Placeat', 'Alimentos', 232.45);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (58, 'Recusandae Voluptas', 'Bebidas', 30.82);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (59, 'Eum Voluptate', 'Livros', 237.81);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (60, 'Nulla Sit', 'Alimentos', 17.12);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (61, 'Fugiat Omnis', 'Eletrônicos', 236.65);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (62, 'Ipsum Commodi', 'Eletrônicos', 861.04);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (63, 'Consequatur At', 'Eletrônicos', 519.01);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (64, 'Eaque Illum', 'Bebidas', 672.29);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (65, 'Assumenda Totam', 'Alimentos', 543.84);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (66, 'Tempore Vero', 'Livros', 580.45);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (67, 'Delectus Vel', 'Alimentos', 786.77);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (68, 'Facere Sit', 'Roupas', 198.51);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (69, 'Laboriosam Harum', 'Eletrônicos', 662.39);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (70, 'Fugit Magnam', 'Bebidas', 429.34);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (71, 'Ex Quae', 'Roupas', 865.2);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (72, 'Ab Nulla', 'Eletrônicos', 676.63);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (73, 'Incidunt Quaerat', 'Eletrônicos', 70.01);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (74, 'Ullam Assumenda', 'Bebidas', 802.59);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (75, 'Deleniti Aperiam', 'Eletrônicos', 256.17);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (76, 'Inventore Quae', 'Alimentos', 540.92);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (77, 'Illum Laborum', 'Alimentos', 427.66);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (78, 'Ducimus Ipsum', 'Bebidas', 468.0);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (79, 'Maiores Cum', 'Eletrônicos', 448.7);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (80, 'Quos Cumque', 'Livros', 106.93);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (81, 'Optio Exercitationem', 'Livros', 837.67);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (82, 'Distinctio Vel', 'Eletrônicos', 927.1);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (83, 'Dolore Repellendus', 'Alimentos', 174.65);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (84, 'Commodi Quisquam', 'Roupas', 486.54);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (85, 'Beatae Iste', 'Roupas', 903.42);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (86, 'Aliquid Magnam', 'Alimentos', 385.18);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (87, 'Reiciendis Commodi', 'Roupas', 272.55);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (88, 'Possimus Doloremque', 'Roupas', 292.4);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (89, 'Molestiae Ad', 'Livros', 665.25);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (90, 'Dolorum Earum', 'Roupas', 163.25);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (91, 'Saepe Voluptatum', 'Bebidas', 225.52);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (92, 'Voluptatum Magni', 'Eletrônicos', 583.39);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (93, 'Sapiente Et', 'Livros', 70.35);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (94, 'Quibusdam Quo', 'Bebidas', 66.59);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (95, 'Qui Accusamus', 'Livros', 482.02);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (96, 'Eaque Adipisci', 'Livros', 165.86);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (97, 'Repellendus Iste', 'Livros', 89.31);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (98, 'Nisi Iste', 'Alimentos', 77.84);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (99, 'Eum Quisquam', 'Eletrônicos', 678.46);
INSERT INTO dim_produto (id_produto, nome_produto, categoria, preco) VALUES (100, 'Amet Occaecati', 'Alimentos', 409.74);
                
                
"""]

# Executar a criação das tabelas
with engine.begin() as conn:
  for cmd in ddl_dim_fato:
    conn.execute(text(cmd))
  print("Dados inseridos!")
  

Dados inseridos!


In [ ]:
ddl_dim_fato = ["""

INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (1, 73, 32, '2024-12-02', 5, 2992.85);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (2, 80, 11, '2024-08-15', 4, 2643.2);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (3, 73, 67, '2025-01-27', 3, 2806.08);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (4, 27, 86, '2025-01-20', 3, 738.9);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (5, 51, 17, '2024-07-20', 3, 1387.95);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (6, 97, 10, '2025-03-02', 1, 463.7);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (7, 73, 13, '2024-08-21', 1, 542.26);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (8, 65, 34, '2025-02-25', 2, 1867.86);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (9, 9, 32, '2024-10-17', 3, 876.42);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (10, 57, 70, '2024-06-10', 3, 1846.68);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (11, 84, 68, '2025-01-06', 1, 671.18);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (12, 71, 39, '2024-10-21', 1, 939.54);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (13, 18, 34, '2025-04-02', 1, 890.82);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (14, 96, 71, '2025-01-17', 2, 559.24);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (15, 78, 27, '2025-02-21', 3, 634.68);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (16, 82, 34, '2024-10-18', 5, 2468.25);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (17, 7, 12, '2024-09-01', 4, 3324.2);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (18, 6, 1, '2024-06-18', 3, 2320.23);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (19, 82, 34, '2024-09-25', 2, 1487.64);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (20, 71, 91, '2024-11-15', 4, 2261.2);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (21, 15, 10, '2024-08-23', 2, 1100.26);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (22, 48, 75, '2025-02-21', 5, 783.05);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (23, 17, 6, '2024-11-30', 3, 1113.03);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (24, 6, 46, '2025-05-13', 2, 1370.5);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (25, 86, 14, '2025-01-26', 3, 2346.96);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (26, 53, 80, '2025-01-12', 2, 1853.02);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (27, 31, 21, '2025-02-11', 2, 1765.2);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (28, 4, 23, '2025-06-09', 3, 2353.68);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (29, 53, 86, '2025-06-09', 2, 548.28);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (30, 90, 14, '2024-07-04', 4, 3493.8);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (31, 61, 29, '2025-03-05', 2, 1636.84);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (32, 59, 45, '2025-01-02', 3, 2466.99);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (33, 30, 29, '2024-06-16', 1, 663.43);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (34, 52, 43, '2025-04-14', 3, 2597.13);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (35, 99, 36, '2024-12-01', 3, 1935.21);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (36, 52, 87, '2025-01-17', 5, 1689.2);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (37, 4, 15, '2024-06-23', 3, 560.34);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (38, 34, 5, '2025-01-31', 1, 600.61);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (39, 45, 94, '2025-02-01', 3, 1326.09);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (40, 66, 15, '2024-11-02', 4, 3602.04);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (41, 25, 33, '2024-11-23', 1, 711.7);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (42, 1, 67, '2024-12-09', 5, 3449.95);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (43, 95, 95, '2024-11-09', 2, 741.16);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (44, 9, 86, '2024-09-25', 3, 1880.76);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (45, 85, 16, '2024-10-09', 3, 1536.15);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (46, 86, 53, '2024-09-17', 3, 1225.14);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (47, 38, 71, '2025-05-23', 2, 399.84);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (48, 86, 49, '2025-01-31', 2, 1238.72);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (49, 39, 52, '2024-08-24', 5, 4176.95);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (50, 39, 37, '2025-01-15', 2, 871.18);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (51, 75, 78, '2025-03-28', 3, 1411.02);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (52, 57, 87, '2024-12-15', 2, 1032.12);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (53, 95, 22, '2024-09-03', 1, 290.95);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (54, 85, 82, '2025-04-24', 5, 1709.2);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (55, 97, 31, '2024-09-08', 3, 697.17);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (56, 26, 19, '2024-09-12', 1, 55.75);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (57, 61, 79, '2025-03-23', 1, 460.86);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (58, 81, 74, '2025-01-18', 2, 1442.32);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (59, 50, 64, '2025-04-01', 4, 1006.16);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (60, 84, 89, '2024-12-28', 1, 893.73);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (61, 99, 14, '2024-11-28', 4, 906.6);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (62, 90, 67, '2024-10-14', 4, 238.88);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (63, 32, 16, '2024-09-10', 4, 568.04);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (64, 60, 86, '2025-04-20', 5, 4940.05);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (65, 77, 41, '2024-08-07', 4, 2466.12);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (66, 93, 65, '2024-07-27', 4, 3329.04);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (67, 71, 58, '2024-08-08', 2, 1492.44);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (68, 61, 58, '2024-10-15', 3, 2262.78);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (69, 82, 36, '2025-03-25', 5, 2448.75);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (70, 31, 36, '2024-10-12', 4, 346.84);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (71, 37, 31, '2025-03-06', 3, 1027.47);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (72, 70, 11, '2024-10-06', 2, 318.64);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (73, 50, 89, '2025-04-14', 2, 1418.7);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (74, 9, 54, '2025-05-31', 4, 1350.28);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (75, 60, 54, '2024-10-09', 1, 214.77);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (76, 54, 50, '2024-10-14', 5, 4733.0);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (77, 3, 98, '2024-12-20', 5, 1932.9);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (78, 1, 46, '2024-09-14', 3, 2267.82);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (79, 54, 69, '2024-12-16', 5, 4009.35);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (80, 29, 63, '2024-09-24', 2, 560.42);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (81, 63, 4, '2024-09-08', 4, 1371.08);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (82, 87, 52, '2024-08-18', 2, 1684.2);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (83, 17, 80, '2024-11-13', 5, 183.5);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (84, 51, 76, '2025-05-25', 5, 3331.85);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (85, 11, 83, '2024-10-08', 4, 577.36);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (86, 60, 24, '2025-04-12', 1, 267.55);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (87, 42, 28, '2024-10-21', 4, 1334.32);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (88, 98, 49, '2024-10-18', 3, 2263.47);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (89, 54, 33, '2025-02-27', 1, 475.6);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (90, 96, 70, '2025-03-30', 1, 999.91);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (91, 45, 29, '2024-12-17', 1, 783.42);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (92, 84, 6, '2025-04-24', 1, 950.12);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (93, 26, 3, '2025-02-05', 5, 804.3);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (94, 17, 61, '2024-11-15', 1, 568.33);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (95, 28, 60, '2024-09-15', 3, 2307.69);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (96, 22, 78, '2024-11-12', 5, 4820.5);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (97, 92, 15, '2025-03-15', 2, 1930.14);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (98, 14, 75, '2024-07-07', 1, 929.6);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (99, 74, 87, '2024-07-20', 4, 1610.76);
INSERT INTO fato_pedidos (id_pedido, id_cliente, id_produto, data_pedido, quantidade, valor_total) VALUES (100, 92, 26, '2025-02-07', 1, 596.18); 
                
"""]

# Executar a criação das tabelas
with engine.begin() as conn:
  for cmd in ddl_dim_fato:
    conn.execute(text(cmd))
  print("Dados inseridos!")
  

Dados inseridos!


### Processamento

Configurações

In [ ]:
# postgreSQL
pg_config = {
  "host": "bd-relacional.cjeco8miq291.us-east-1.rds.amazonaws.com",
  "database":"postgres",
  "user":"postgres",
  "password":"",
  "port":5432
}


# AWS S3
bucket_name = "323317787666"
s3_prefix = "raw/"

# nome das tabelas
tabelas = ['dim_cliente', 'dim_produto', 'fato_pedidos']

In [3]:
# conexão com o postgress
conn = psycopg2.connect(**pg_config)

In [4]:
# AWS S3
bucket_name = "323317787666"
s3_prefix = "raw/"

In [5]:
# conexão com S3
s3 = boto3.client('s3')
region = s3.meta.region_name or "us-east-1"

In [6]:
# verifica se já existe o bucket criado, se não, realiza a criação
try:
  s3.head_bucket(Bucket=bucket_name)
  print(f"Bucket {bucket_name} já existe")
except ClientError as e:
  error_code = int(e.response['Error']['Code'])
  if error_code == 404:
    print(f"Bucket {bucket_name} não existe, criando...")
    if region == "us-east-1":
      s3.create_bucket(Bucket=bucket_name)
    else:
      s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
      )
    print(f"Bucket {bucket_name} criado com sucesso")
  else:
    raise


Bucket 323317787666 já existe


In [7]:
# exportação das tabelas
for tabela in tabelas:
  print(f"Exportando tabela: {tabela}")
  df = pd.read_sql(f"SELECT * FROM {tabela};", conn)

  # salvar como csv em memória
  csv_buffer = StringIO()
  df.to_csv(csv_buffer, index=False)

  # Enviar para S3
  s3_key = f"{s3_prefix}{tabela}.csv"
  s3.put_object(Bucket=bucket_name, Key=s3_key, Body=csv_buffer.getvalue())
  print(f"{tabela} salva no S3 em: s3://{bucket_name}/{s3_key}")

# fecha conexão 
conn.close
print("Exportação finalizada")

Exportando tabela: dim_cliente


C:\Users\gabri\AppData\Local\Temp\ipykernel_2656\1993591939.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {tabela};", conn)


dim_cliente salva no S3 em: s3://323317787666/raw/dim_cliente.csv
Exportando tabela: dim_produto
dim_produto salva no S3 em: s3://323317787666/raw/dim_produto.csv
Exportando tabela: fato_pedidos
fato_pedidos salva no S3 em: s3://323317787666/raw/fato_pedidos.csv
Exportação finalizada


In [24]:
RAW_PREFIX = f's3://{bucket_name}/raw/'
SILVER_PREFIX = f's3://{bucket_name}/silver/' 
GOLD_PREFIX = f's3://{bucket_name}/gold' 

In [17]:
tabelas

['dim_cliente', 'dim_produto', 'fato_pedidos']

In [25]:
# Camada Silver
clientes = pd.read_csv(RAW_PREFIX + 'dim_cliente.csv')
produto = pd.read_csv(RAW_PREFIX + 'dim_produto.csv')
pedidos = pd.read_csv(RAW_PREFIX + 'fato_pedidos.csv')

In [26]:
# padronizar colunas para lower case
for df in [clientes, produto, pedidos]:
  df.columns = [col.lower() for col in df.columns]

In [27]:
# limpeza básica
clientes = clientes.drop_duplicates().dropna(subset=['id_cliente'])
produto = produto.drop_duplicates().dropna(subset=['id_produto'])
pedidos = pedidos.drop_duplicates().dropna(subset=['id_cliente', 'id_produto'])

In [36]:
pedidos.head(10)

,id_pedido,id_cliente,id_produto,data_pedido,quantidade,valor_total
0,1,73,32,2024-12-02,5,2992.85
1,2,80,11,2024-08-15,4,2643.20
2,3,73,67,2025-01-27,3,2806.08
3,4,27,86,2025-01-20,3,738.90
4,5,51,17,2024-07-20,3,1387.95
5,6,97,10,2025-03-02,1,463.70
6,7,73,13,2024-08-21,1,542.26
7,8,65,34,2025-02-25,2,1867.86
8,9,9,32,2024-10-17,3,876.42
9,10,57,70,2024-06-10,3,1846.68


In [37]:
produto.head(20)

,id_produto,nome_produto,categoria,preco
0,1,Iste Illum,Alimentos,171.77
1,2,Libero Enim,Roupas,277.25
2,3,Quis At,Livros,227.42
3,4,Soluta At,Bebidas,844.42
4,5,Cupiditate Nostrum,Eletrônicos,236.76
5,6,Sunt Voluptas,Eletrônicos,807.00
6,7,Rerum Explicabo,Roupas,275.06
7,8,Iste Ipsum,Alimentos,914.00
8,9,Sed Reprehenderit,Livros,877.60
9,10,Quo Culpa,Bebidas,220.50


In [31]:
clientes.head()

,id_cliente,nome,idade,cidade
0,1,Daniela da Cunha,58,Dias
1,2,Marcos Vinicius da Mota,25,Costela
2,3,Vitor Nascimento,19,da Paz
3,4,Dra. Caroline Nascimento,65,Mendes da Prata
4,5,Igor Rezende,35,Nunes


In [29]:
# padronização das colunas datas
if 'data_pedido' in pedidos.columns:
  pedidos['data_pedido'] = pd.to_datetime(pedidos['data_pedido'], errors='coerce')

In [34]:
# salvar camada silver em parquet no S3
clientes.to_parquet(SILVER_PREFIX + 'clientes/', index=False)
produto.to_parquet(SILVER_PREFIX + 'produto/', index=False)
pedidos.to_parquet(SILVER_PREFIX + 'pedidos/', index=False)

In [35]:
pedidos.head()

,id_pedido,id_cliente,id_produto,data_pedido,quantidade,valor_total
0,1,73,32,2024-12-02,5,2992.85
1,2,80,11,2024-08-15,4,2643.20
2,3,73,67,2025-01-27,3,2806.08
3,4,27,86,2025-01-20,3,738.90
4,5,51,17,2024-07-20,3,1387.95


In [ ]:
# gold layer
gold_vendas = pedido.merge()